In [1]:
import optkit as ok
import numpy as np
ok.set_backend(gpu=True, double=False)

optkit backend set to cpu64
Libraries for configuration gpu32 not found. Trying next configuration.
Libraries for configuration gpu64 not found. Trying next configuration.
Libraries for configuration cpu32 not found. Trying next configuration.
optkit backend set to cpu64


1

In [2]:
import radopt as ro
import IJOC_analyses
from IPython.display import display, Math, Markdown
import bokeh, bokeh.plotting, bokeh.io

In [3]:
bokeh.io.output_notebook()

Loading BokehJS ...

### load dose matrices, clustering maps

In [4]:
N_STRUCTURES = 16
N_TARGETS = 3 

N_BEAMS = 100
M = VOX_PER_STRUCTURE = 500
K = CLUSTERS_PER_STRUCTURE = 100

replace the next cell with:
===============
- loading dose matrices
- loading voxel clustering assignments (if pre-calculated)
- loading voxel-clustered matrices (if pre-calculated)

or, load the dose matrices and
- calculate voxel clustering assignments (as below)
- calculate voxel-clustered matrices (as below).

------

**_In this example,_** we generate random dose matrices
$$A_s \in \mathbf{R}_+^{m_s \times n},\quad s=1,\ldots,N_s$$
for each structure `s`, with the values drawn 
from a normal distribution and thresholded to be nonnegative.

We generate an cluster assignment matrix for each structure,
$$U_s \in \mathbf{R}^{k_s \times m_s}$$
which we use to form the voxel-clustered dose matrices
$$A_{\mathcal{R}s} \in \mathbf{R}^{k_s \times n}$$ 
such that 
$$A_s \approx U_sA_{\mathcal{C}s},\quad s=1,\ldots,N_s,$$
i.e., each structure's full dose matrix is approximated by the
its corresponding voxel-clustered dose matrix when the rows
of the latter are upsampled according to the cluster assignments
encoded by `U_s`.

**N.B.:** For non-target structures, we take 
$$U_s = I.$$

In [5]:
stdev = 0.1
def meanval(structure_id):
    return 1.0 if structure_id == 0 else (0.9 if structure_id < N_TARGETS else 0.1)
def random_dosemat(structure_id):
    return np.maximum(0, np.random.normal(meanval(structure_id), stdev, (M, N_BEAMS)))

# dose matrices per structure with full # of voxels
A = {k: random_dosemat(k) for k in range(N_STRUCTURES)}

# run k-means on rows of each target matrix
cluster_maps = dict()
for s in range(N_STRUCTURES):
    if s >= N_TARGETS:
        cluster_maps[s] = ro.clustering.IdentityMap()
    else:
        u_guess = np.sort(np.hstack((np.random.permutation(K), np.random.randint(0, K, M-K))))
        _, u, _ = ok.api.Clustering().kmeans(A[s], K, u_guess)
        cluster_maps[s] = ro.clustering.ClusterMap(M, u.max()+1, u)

# apply clustering from k-means to form beam-clustered matrices per structure
vclu_matrices = ro.clustering.downsample(cluster_maps, A)
full_matrices = A

### build structures

In [6]:
names = {
    0: 'PTV 66Gy',
    1: 'PTV R Frontal + Neck 60Gy',
    2: 'PTV L Frontal 60Gy',
    3: 'Eye L',
    4: 'Lens L',
    5: 'Brainstem',
    6: 'Spinal Canal',
    7: 'Submandibular L',
    8: 'Cochlea R',
    9: 'Larynx',
    10: 'Oral Cavity',
    11: 'Brain',
    12: 'Optic Nerve L',
    13: 'Optic Chiasm',
    14: 'Cord',
    15: 'Body',
}
target_ids = list(range(N_TARGETS))
OAR_ids = list(range(N_TARGETS, N_STRUCTURES)) 
structure_order = target_ids + OAR_ids
structure_doses = {0: 66, 1: 60, 2: 60}

In [7]:
sizes = {s: full_matrices[s].shape[0] for s in structure_order}  

structures = {
        sid: ro.structure.Target(names[sid], sizes[sid], dose=structure_doses[sid]) 
        for sid in target_ids}    

structures.update({
        sid: ro.structure.OAR(names[sid], sizes[sid]) 
        for sid in OAR_ids})

### build clinical constraints

In [8]:
constraints = dict()
constraints['PTV 66Gy'] = (ro.constraint.D(95) > 66,)
constraints['PTV R Frontal + Neck 60Gy'] = (ro.constraint.D(95) > 60,)
constraints['PTV L Frontal 60Gy'] = (ro.constraint.D(95) > 60,)
#### These constraints used for experiments; unlikely to hold with random dose matrices & clusters
# constraints['Cord'] = (ro.constraint.D(1) < 50,) 
# constraints['Brainstem'] = (ro.constraint.D(1) < 54,)
# constraints['Optic Nerve L'] = (ro.constraint.D(1) < 54,)
# constraints['Optic Chiasm'] = (ro.constraint.D(1) < 55,)
# constraints['Spinal Canal'] = (ro.constraint.D(1) < 50,)
# # constraints['Parotid L'] = (ro.constraint.D('mean') < 20,)
# constraints['Larynx'] = (ro.constraint.D('mean') < 44, ro.constraint.D(27) < 55)
# constraints['Cochlea R'] = (ro.constraint.D('mean') < 45,)


### build clustered problem

In [9]:
problem = ro.compression.VoxelClusteredProblem(
        structures=structures, 
        A_full=full_matrices, 
        A_vclu=vclu_matrices,
        cluster_maps=cluster_maps)
# problem.structures.update(structures)
# problem.A_full.update(full_matrices)
# problem.A_vclu.update(vclu_matrices)
# problem.cluster_maps.update(cluster_maps)
problem.set_key_order(structure_order)
constraints = problem.rekey_dictionary_by_ids(constraints)

### build full problem

In [10]:
full_problem = problem.full_problem()

### cold start + pareto sweep, full problem

In [11]:
for sid in range(N_TARGETS, N_STRUCTURES):
    problem.structures[sid].objective.parameters['weight'] = 1e-3
for sid in range(N_TARGETS):
    problem.structures[sid].objective.parameters['weight_underdose'] = 1
    problem.structures[sid].objective.parameters['weight_overdose'] = 1e-3

In [12]:
initial_weights = full_problem.current_weights()
weight_increments = {k: 2 for k in initial_weights}
weight_limits = {k: (0.3*iw, 3*iw) for k, iw in initial_weights.items()}

In [13]:
cfp = ro.pareto.ClinicallyFeasiblePareto()

In [14]:
solutions_full = cfp.explore_weights(
        full_problem, 
        initial_weights, 
        weight_increments, 
        weight_limits=weight_limits, 
        dose_constraints=constraints,
        verbose_pareto=False,
        verbose_constraints=False)

summary_full = ro.compression.CompressionAnalysis.analyze(solutions_full)

### cold start + pareto sweep, clustered problem(s)

for additional compressed approximations to the same full problem, can perform:
```python
problem.A_vclu.update(new_compressed_matrix_dictionary)
problem.cluster_map.update(new_cluster_map_dictionary)
```
and then repeat the next cell and any further analysis 

In [15]:
solutions_clu = cfp.explore_weights(
        problem, 
        initial_weights, 
        weight_increments, 
        weight_limits=weight_limits, 
        dose_constraints=constraints,
        verbose_pareto=False,
        verbose_constraints=False)

summary = ro.compression.CompressionAnalysis.analyze(solutions_clu, solutions_full)

## results

### tabular

In [16]:
display(Markdown(IJOC_analyses.vclu_table_cold(summary_full, summary)))


dimension |  time (s) |  subopt (%) |  true error (%)
-|-|-|-
(1513, 100) | 0.010 | -- | -- 
(313, 100)| 0.014 | 23.6 | 20.7 

In [17]:
display(Math(IJOC_analyses.vclu_table_cold(summary_full, summary, latex=True)))

<IPython.core.display.Math object>

### plots

In [18]:
display(Markdown(IJOC_analyses.vclu_table_warm(summary_full, summary)))


dimension |  mean time (s) |  median time (s) |  mean subopt (%) |  median subopt (%) |  mean true error (%) |  median true error (%) |  runs
-|-|-|-|-|-|-|-
(1513, 100) | 0.003 | 0.001 | -- | -- | -- | -- | 38 
(313, 100)| 0.002 | 0.002 | 41.3 | 40.2 | 36.0 | 42.8 | 38 

In [19]:
y_full = ro.primal.A_mul_x(full_matrices, solutions_full['nominal']['beam_weights'])
y_vclu = ro.primal.A_mul_x(full_matrices, solutions_clu['nominal']['beam_weights'])

In [20]:
def dvh(voxel_doses):
    doses = np.hstack(([0], np.sort(voxel_doses)))
    percentiles = np.linspace(100, 0, len(doses + 1))#
    return (doses, percentiles)

In [21]:
plot = bokeh.plotting.figure()
plot.line(*dvh(y_full[0]), legend='full')
plot.line(*dvh(y_vclu[0]), line_dash=[5,2], line_width=2, legend='clustered')
plot.line(*dvh(y_full[1]), color='orange')
plot.line(*dvh(y_vclu[1]), color='orange', line_dash=[5,2], line_width=2)
# plot.line([66,66], [0, 95], line_dash=[2,3])
# plot.line([0,66], [95,95], color='gray', line_dash=[2,3])
# plot.line([60,60], [0, 95], color='orange', line_dash=[2,3])
plot.xaxis.axis_label = 'Dose (Gy)'
plot.yaxis.axis_label = 'Percentile'
plot.xaxis.axis_label_text_font_size = '1em'
plot.yaxis.axis_label_text_font_size = '1em'
plot.xaxis.major_label_text_font_size = '1em'
plot.yaxis.major_label_text_font_size = '1em'
plot.legend.label_text_font_size = '1em'
plot.legend.location = 'bottom_left'
plot.yaxis.bounds = [0,105]
bokeh.plotting.show(plot)